In [2]:
import string
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re

#nltk.download('stopwords')

# Load stop words liss
stop_words = stopwords.words('english')
with open("C:/Users/rutup/Desktop/Data_Prrojects/Blackcoffer/StopWords/All_Words.txt", 'r') as file:
    stop_words2 = file.read().splitlines()
stop_words+=stop_words2

stemmer = SnowballStemmer("english")


# Remove stop words
filtered_tokens = [token for token in tokens if token.lower() not in [stop_words,stemmer.stem(token)]]

# Join filtered tokens back into a string
filtered_text = ' '.join(filtered_tokens)

print(filtered_text)

In [22]:
def preprocess(text_o, stem=False):
    # Remove link,user and special characters
    text = re.sub(r'[^\w\s]+|(\s)#\w+', '', text_o).lower().strip()
    # Remove quotation marks
    #text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)

    print((" ".join(tokens)), '\n')
    print(text)
    return (" ".join(tokens)), text

#f_text, text = preprocess(text_o)
#f_text

### Positive words and negetive words 

In [23]:
def positive_negetive():
    words = f_text.split()

    positive_words = []
    negative_words = []

    with open("positive-words.txt", "r") as f:
        positive_words = f.read().splitlines()

    with open("negative-words.txt", "r") as f:
        negative_words = f.read().splitlines()

    #Check each word and count positive and negative words
    positive_count = 0
    negative_count = 0

    positive_dict = {}
    negative_dict = {}

    total_chars = 0

    for word in words:
        total_chars += len(word)

        if word in positive_words:
            positive_count += 1
            if word in positive_dict:
                positive_dict[word] += 1
            else:
                positive_dict[word] = 1
        elif word in negative_words:
            negative_count += 1
            if word in negative_dict:
                negative_dict[word] += 1
            else:
                negative_dict[word] = 1

    polarity_score = (positive_count - negative_count)/((positive_count+negative_count)+0.000001)

    # Step 4: Print the results
    #print("Positive words found:", positive_count)
    #print("Positive word dictionary:", positive_dict)
    #print("Negative words found:", negative_count)
    #print("Negative word dictionary:", negative_dict)
    #print("Polary score = ", polarity_score)

    word_count = len(nltk.word_tokenize(f_text))
    #print('word count= ',word_count)

    sub_score = (positive_count+negative_count)/((word_count)+0.000001)

    #print('Subjectivity Score= ', sub_score)

    avg_word_length = total_chars / len(words)
    
    return positive_count, negative_count, polarity_score, sub_score, word_count,avg_word_length

#positive_count, negative_count, polarity_score, sub_score, word_count = positive_negetive()


### Complex Word

In [24]:
def comp_word():
    from nltk.corpus import cmudict

    # Load the CMU Pronouncing Dictionary
    cmud = cmudict.dict()

    # Tokenize the text into words
    words = nltk.word_tokenize(f_text)

    # Function to count the number of syllables in a word
    def count_syllables(word):
        """
        Count the number of syllables in a word using the CMU Pronouncing Dictionary.
        """
        if word.lower() in cmud:
            return max([len(list(y for y in x if y[-1].isdigit())) for x in cmud[word.lower()]])
        else:
            return 0
        
    syllables_per_word = [count_syllables(word) for word in words]

    # Calculate the average number of syllables per word
    syllables_count = sum(syllables_per_word) / len(syllables_per_word)

    # Filter out words with less than 3 syllables
    complex_words = [word for word in words if count_syllables(word) > 2]

    #print("Complex words:", complex_words)

    return len(complex_words), syllables_count

#complex_word = comp_word()
#print("Complex words:", complex_word)

### Analysis of readability

In [25]:
def readability():
    sen_length = word_count / len(nltk.sent_tokenize(text_o))

    comp_percent = complex_word / word_count

    Fog_Index = 0.4*(sen_length + comp_percent)

    return sen_length,comp_percent,Fog_Index

#sen_length,comp_percent,Fog_Index = readability()

### Pronoun Count

In [26]:
def pronoun():
    # Define a regular expression pattern that matches the specified pronouns
    pattern = r'\b(?<!U)I\b|\b(?<!U)we\b|\bmy\b|\bours\b|\bus\b'

    matches = re.findall(pattern, text_o)

    # Count the number of matches for each pronoun
    counts = {
        "I": 0,
        "we": 0,
        "my": 0,
        "ours": 0,
        "us": 0
    }

    for match in matches:
        if match == "I":
            counts["I"] += 1
        elif match == "we":
            counts["we"] += 1
        elif match == "my":
            counts["my"] += 1
        elif match == "ours":
            counts["ours"] += 1
        elif match == "us":
            counts["us"] += 1

    #print
    #print("Counts of personal pronouns:")
    #for pronoun, count in counts.items():
        #print(pronoun, count)

    pronouns = sum(counts.values())
    
    return pronouns

#pronouns = pronoun()


In [36]:
import pandas as pd

l1= {'id':[],'POSITIVE SCORE':[], 'NEGATIVE SCORE':[], 'POLARITY SCORE':[], 'SUBJECTIVITY SCORE':[],
     'AVG SENTENCE LENGTH':[], 'PERCENTAGE OF COMPLEX WORDS':[], 'FOG INDEX':[],'AVG NUMBER OF WORDS PER SENTENCE':[],
     'COMPLEX WORD COUNT':[], 'WORD COUNT':[], 'SYLLABLE PER WORD':[], 'PERSONAL PRONOUNS':[], 'AVG WORD LENGTH':[]}
#df1 = pd.DataFrame(columns = [i,positive_count, negative_count, polarity_score, sub_score, comp_percent, Fog_Index, sen_length, complex_word, word_count])

for i in range(37,151):
    if i not in [44,57,144]: 
        # Example text
        print(i)
        path = "C:/Users/rutup/Desktop/Data_Prrojects/Blackcoffer/text/"+str(i)+".txt"
        with open(path, 'r') as file:
            text_o = file.read()

        #text = re.sub(text, ' ', str(text).lower()).strip()

        # Tokenize text
        tokens = nltk.word_tokenize(text_o)
        
        f_text, text = preprocess(text_o)
        positive_count, negative_count, polarity_score, sub_score, word_count, avg_word_length = positive_negetive()
        complex_word, syllables_count = comp_word()
        sen_length,comp_percent,Fog_Index = readability()
        pronouns = pronoun()

        l1['id'].append(i), l1['POSITIVE SCORE'].append(positive_count)
        l1['NEGATIVE SCORE'].append(negative_count),l1['POLARITY SCORE'].append(polarity_score), l1['SUBJECTIVITY SCORE'].append(sub_score)
        l1['AVG SENTENCE LENGTH'].append(sen_length),l1['PERCENTAGE OF COMPLEX WORDS'].append(comp_percent), l1['FOG INDEX'].append(Fog_Index)
        l1['AVG NUMBER OF WORDS PER SENTENCE'].append(sen_length),l1['COMPLEX WORD COUNT'].append(complex_word),l1['WORD COUNT'].append(word_count)
        l1['SYLLABLE PER WORD'].append(syllables_count)
        l1['PERSONAL PRONOUNS'].append(pronouns),l1['AVG WORD LENGTH'].append(avg_word_length)

37
ai healthcare improve patient outcomes introduction kills 10 million people decades highly infectious virus war missiles microbes bill gatess remarks ted conference 2014 world avoided ebola outbreak unprecedented invisible virus hit met overwhelmed unprepared healthcare system oblivious population public health emergency demonstrated lack scientific consideration underlined alarming robust innovations health medical facilities past years artificial intelligence proven tangible potential healthcare sectors clinical practices translational medical biomedical research case detected china december 31st 2019 ai program developed bluedot alerted world pandemic quick realise ais ability analyse large chunks data detecting patterns identifying tracking carriers virus tracing apps ai tabs people infected prevent risk crossinfection ai algorithms track patterns extract features classify categorise ai ibm watson sophisticated ai works cloud computing natural language processing prominently con

In [37]:
df2 = pd.read_excel('Output Data Structure.xlsx')
df2 = df2.iloc[:,:2]

In [38]:
df1 = pd.DataFrame.from_dict(l1)

In [39]:
df = pd.merge(df2, df1, left_on='URL_ID', right_on='id', how='left')
del df['id']

In [40]:
df.to_excel('output.xlsx', index=False)